In [ ]:
import numpy as np

# Entrada/Salida de datos
data = open('input.txt', 'r').read() # archivo de texto plano de entrada
chars = list(set(data)) # caracteres unicos en el archivo de entrada
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) } # diccionario: caracter -> indice
ix_to_char = { i:ch for i,ch in enumerate(chars) } # diccionario: indice -> caracter

# Hiperparametros
hidden_size = 100 # tamaño de la capa oculta (cantidad de neuronas)
seq_length = 25 # numero de pasos para desenrollar la RNN (longitud de secuencia)
learning_rate = 1e-1

# parametros del modelo (matrices de pesos y sesgos)
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # pesos: entrada -> capa oculta
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # pesos: capa oculta -> capa oculta (memoria)
Why = np.random.randn(vocab_size, hidden_size)*0.01 # pesos: capa oculta -> salida
bh = np.zeros((hidden_size, 1)) # sesgo de la capa oculta
by = np.zeros((vocab_size, 1)) # sesgo de la capa de salida

def lossFun(inputs, targets, hprev):
  """
  Calcula la pérdida (loss) y los gradientes para actualizar los pesos.
  
  Parámetros:
  - inputs: lista de índices de caracteres de entrada
  - targets: lista de índices de caracteres objetivo (siguiente carácter)
  - hprev: estado oculto anterior (memoria de la red)
  
  Retorna:
  - loss: valor de la función de pérdida
  - gradientes para cada parámetro (dWxh, dWhh, dWhy, dbh, dby)
  - último estado oculto
  """
  xs, hs, ys, ps = {}, {}, {}, {} # diccionarios para almacenar valores en cada paso temporal
  hs[-1] = np.copy(hprev) # inicializar con el estado oculto anterior
  loss = 0
  
  # Propagacion hacia adelante (forward pass)
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # codificar entrada en representacion one-hot
    xs[t][inputs[t]] = 1 # poner un 1 en la posicion del caracter actual
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # calcular estado oculto
    ys[t] = np.dot(Why, hs[t]) + by # calcular salida (logits sin normalizar)
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # aplicar softmax: convertir a probabilidades
    loss += -np.log(ps[t][targets[t],0]) # calcular perdida con entropia cruzada
  
  # Propagacion hacia atras (backward pass): calcular gradientes en orden inverso
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0]) # gradiente del siguiente estado oculto
  
  for t in reversed(range(len(inputs))): # recorrer desde el final hasta el inicio
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # gradiente de la funcion softmax con entropia cruzada
    dWhy += np.dot(dy, hs[t].T) # gradiente de Why
    dby += dy # gradiente de by
    dh = np.dot(Why.T, dy) + dhnext # gradiente que llega a la capa oculta
    dhraw = (1 - hs[t] * hs[t]) * dh # gradiente a traves de la funcion tanh
    dbh += dhraw # gradiente de bh
    dWxh += np.dot(dhraw, xs[t].T) # gradiente de Wxh
    dWhh += np.dot(dhraw, hs[t-1].T) # gradiente de Whh
    dhnext = np.dot(Whh.T, dhraw) # gradiente para el paso temporal anterior
  
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # recortar gradientes para evitar explosion
  
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  Genera una secuencia de caracteres del modelo entrenado.
  
  Parámetros:
  - h: estado oculto (memoria) inicial
  - seed_ix: índice del carácter semilla para comenzar
  - n: cantidad de caracteres a generar
  
  Retorna:
  - lista de índices de caracteres generados
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1 # codificar caracter inicial en one-hot
  ixes = []
  
  for t in range(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh) # actualizar estado oculto
    y = np.dot(Why, h) + by # calcular salida
    p = np.exp(y) / np.sum(np.exp(y)) # convertir a probabilidades con softmax
    ix = np.random.choice(range(vocab_size), p=p.ravel()) # muestrear siguiente caracter
    x = np.zeros((vocab_size, 1))
    x[ix] = 1 # actualizar entrada para el siguiente paso
    ixes.append(ix)
  
  return ixes

# Variables para el entrenamiento
n, p = 0, 0 # n: contador de iteraciones, p: puntero de posición en los datos
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # variables de memoria para Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # perdida inicial estimada

# Bucle de entrenamiento infinito
while True:
  # Preparar entradas (recorremos el texto de izquierda a derecha en pasos de seq_length)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reiniciar memoria de la RNN
    p = 0 # volver al inicio de los datos
  
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]] # secuencia de entrada
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]] # secuencia objetivo (siguiente caracter)

  # Generar texto de muestra cada 100 iteraciones
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200) # generar 200 caracteres
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print('----\n %s \n----' % (txt, ))

  # Calcular perdida y gradientes
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001 # suavizar la perdida para visualizacion
  if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # mostrar progreso
  
  # Actualizar parametros usando Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam # acumular cuadrados de gradientes
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # actualizacion Adagrad

  p += seq_length # mover puntero de datos
  n += 1 # incrementar contador de iteraciones 


data has 58 characters, 21 unique.
----
 RrRibsnNlbtfsorNixRibRdniNcRRRamRTstsTcdiuaduNucxceudmoxttpTpdeRtTpfnmofll uRitTpnmruaNeTxpbcfRreixsopixsRnmsctffndno nmtbmdieob iTNNciRanceuRNb RicaanfardcmRiuslcnissNasrxRnTenrbroabdmbRcbNrN lbNrm   
----
iter 0, loss: 76.113061
----
 exto te a onirtlcfpa eenndonneenna xntplce nitenaoe raeantenda Ra pabauntie i prnamnifenc enata s ona rapd ucdpsce nctenctRarfendereioronee cip a andfpncionce ncfonttentee a elba on enaepaTfondpunnron 
----
iter 100, loss: 74.770845
----
 exta de pruera ettenbmpa elsimple pa funciona plba ertenaruedr simploere fundmmndrp e par  entendeleco simpae para simple para entensee cRma funcioneae prueba fcttnaioeb papae cora funcionbauebde como 
----
iter 200, loss: 69.183198
----
 exto de prueba simple para enteed ple paio simple parapo amo funciona para entender co siisecf scmple para entender como funciona pnbar  funciona parar cornnfunciona plndsara simple para entencer como 
----
iter 300, loss: 62.890405
----
 exto 